In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Paths
DATA_DIR = '/content/drive/MyDrive/SolarGuard/Solar_Panel_Dataset/Faulty_solar_panel'

# Parameters
IMG_SIZE = (128, 128)
BATCH_SIZE = 16
EPOCHS = 10

# Data Generators
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2
)

train_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=2,
    factor=0.5,
    min_lr=1e-6,
    verbose=1
)

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))



# Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(6, activation='softmax')  # 6 classes
])

base_model.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
history = model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS,class_weight=class_weights,callbacks=[lr_scheduler])

# Save Model
model.save('/content/drive/MyDrive/SolarGuard/mobilenetv2_solar.keras')


Found 743 images belonging to 6 classes.
Found 182 images belonging to 6 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 41s 725ms/step - accuracy: 0.2800 - loss: 1.8839 - val_accuracy: 0.5440 - val_loss: 1.2629 - learning_rate: 0.0010
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 38s 812ms/step - accuracy: 0.5312 - loss: 1.1227 - val_accuracy: 0.6209 - val_loss: 1.0515 - learning_rate: 0.0010
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 31s 665ms/step - accuracy: 0.6709 - loss: 0.8322 - val_accuracy: 0.6429 - val_loss: 0.9199 - learning_rate: 0.0010
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 31s 660ms/step - accuracy: 0.6632 - loss: 0.8386 - val_accuracy: 0.7253 - val_loss: 0.8257 - learning_rate: 0.0010
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 31s 655ms/step - accuracy: 0.6756 - loss: 0.7719 - val_accuracy: 0.7088 - val_loss: 0.8001 - learning_rate: 0.0010
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 522ms/step - accuracy: 0.7480 - loss: 0.6366
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
47/47 ━━━━━━━━━━━━━━━━━━━━ 35s 745ms/step - accuracy: 0.7481 - loss: 0.63

In [6]:
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Get true labels and predictions
val_generator.reset()
pred_probs = model.predict(val_generator, verbose=1)
y_pred = np.argmax(pred_probs, axis=1)
y_true = val_generator.classes

# Class names
class_names = list(val_generator.class_indices.keys())

# Classification report
report = classification_report(y_true, y_pred, target_names=class_names)
print("Classification Report:\n")
print(report)

# Accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.2f}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 568ms/step
Classification Report:

                   precision    recall  f1-score   support

        Bird-drop       0.75      0.59      0.66        41
            Clean       0.79      0.79      0.79        42
            Dusty       0.64      0.66      0.65        38
Electrical-damage       0.90      0.90      0.90        21
  Physical-Damage       0.72      0.81      0.76        16
     Snow-Covered       0.77      0.96      0.85        24

         accuracy                           0.75       182
        macro avg       0.76      0.78      0.77       182
     weighted avg       0.75      0.75      0.75       182

Accuracy: 0.75
